In [1]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import SVC
import os
import matplotlib.pyplot as plt
import time

In [24]:
coin_labels = os.listdir("Data/Train")

In [ ]:
def load_and_process(folder):
    images = []
    labels = []
    for label in coin_labels:
        path = os.path.join(folder, label)
        for filename in os.listdir(path):
            image = cv2.imread(os.path.join(path, filename), cv2.IMREAD_GRAYSCALE)
            image = cv2.medianBlur(image, 5)
            circles = cv2.HoughCircles(
                        image,
                        cv2.HOUGH_GRADIENT,
                        dp=1,
                        minDist=20,
                        param1=100,
                        param2=30,
                        minRadius=100,
                        maxRadius=1000
                    )
            if circles is not None:
                circles = np.int16(np.around(circles))
                x, y, r = circles[0, 0]
                size = r * 2
                x_start = max(0, x-r)
                y_start = max(0, y-r)
                roi = image[y_start:y_start+size, x_start:x_start+size]
                if (roi.shape[0] > 0) and roi.shape[1] > 0:
                    roi = cv2.resize(roi, (256,256))
                    roi = cv2.equalizeHist(roi)
                    images.append(roi)
                    labels.append(label)
            else:
                print("Warning! circles were not found")
                image = cv2.resize(image, (256,256))
                image = cv2.equalizeHist(image)
                images.append(image)
                labels.append(label)
    return images, labels
    

In [26]:
def load_images(folder):
    images = []
    labels = []
    for label in coin_labels:
        path = os.path.join(folder, label)
        for filename in os.listdir(path):
            image = cv2.imread(os.path.join(path, filename), cv2.IMREAD_GRAYSCALE)
            if image is not None:
                images.append(image)
                labels.append(label)
    return images, labels

In [27]:
def rotate_and_save(folder: str):
    labels = os.listdir(folder)
    angles = [cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE, cv2.ROTATE_180]
    for label in labels:
        coins_path = os.path.join(folder, label)
        image_names = os.listdir(coins_path)
        index = len(image_names)
        for image_name in image_names:
            abs_filename = os.path.join(coins_path, image_name)
            img = cv2.imread(abs_filename, cv2.IMREAD_GRAYSCALE)
            for angle in angles:
                rotated = cv2.rotate(img, angle)
                new_filename = os.path.join(coins_path, str(index) + ".png")
                cv2.imwrite(new_filename, rotated)
                index += 1

In [39]:
# loading distinct datasets for training and testing
train_folder = "/home/kolya/code/study/cv/cvlabs/Lab7surf/Data/ClearedTrain"
test_folder = "/home/kolya/code/study/cv/cvlabs/Lab7surf/Data/ClearedTest"

train_images, train_labels = load_images(train_folder)
test_images, test_labels = load_images(test_folder)

In [ ]:
# creating surf descriptors and adding them together
surf = cv2.xfeatures2d.SURF_create(hessianThreshold=200, extended=True)
all_descs = []
for image in train_images:
    _, descriptor = surf.detectAndCompute(image, None)
    print("keypoints: ", len(_))
    all_descs.append(descriptor)
all_descs = np.vstack(all_descs)
print(f"All descriptors size: {all_descs.shape}")

In [41]:
# create vocabulary of descriptors using K-means clusterization
N_CLUSTERS = 1500
descs_clusters = KMeans(n_clusters=N_CLUSTERS, random_state=42).fit(all_descs)

In [42]:
# feature vector is a hitogram of cluserized vocabulary descriptors matches
train_features = []
for image in train_images:
    _, descriptor = surf.detectAndCompute(image, None)
    histogram = np.zeros(N_CLUSTERS)
    labels = descs_clusters.predict(descriptor)
    for label in labels:
        histogram[label] += 1
    train_features.append(histogram / (np.sum(histogram)) + 1e-6) # normalize each histogram
train_features = np.array(train_features)

In [43]:
# create and train SVM classifier with train features and labels
svm = SVC(kernel='rbf', C=5.0, gamma='scale')
svm.fit(train_features, train_labels)

SVC(C=5.0)

In [44]:
# test feature vectors are computed the same way
test_features = []
for image in test_images:
    _, descriptor = surf.detectAndCompute(image, None)
    histogram = np.zeros(N_CLUSTERS)
    labels = descs_clusters.predict(descriptor)
    for label in labels:
        histogram[label] += 1
    test_features.append(histogram / (np.sum(histogram)) + 1e-6) # normalize each histogram
test_features = np.array(test_features)

In [45]:
# predict our results
predictions = svm.predict(test_features)
accuracy = np.mean(predictions == test_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 40.00%
